In [31]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data
import models
from matplotlib_venn import venn3, venn2
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper
import train
from sklearn import decomposition
from sklearn.calibration import calibration_curve
from sklearn.decomposition import PCA, KernelPCA

outcome_def = 'y_consec_thresh'
out_dir = oj('/scratch/users/vision/abc', 'mar7_tf_order_1')
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**baseline stats**

In [32]:
viz.print_metadata(acc=results.accuracy.max())

valid:		1694 aux+ / 6110 (0.277)
----------------------------------------
hotspots:	 295 aux+ /  295
short:		 323 aux+ / 4053 (0.920)
long:		  43 aux+ /   46 (0.935)
hard:		1046 aux+ / 1716 (0.610)
----------------------------------------
hard acc:			  0.738
total acc (no hotspots):	  0.867


**look at prediction metrics**

In [33]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False).head(10)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
model_type,,,,,,
svm_12_None=7_ros=1.2_h=0,0.738,0.731,0.784,0.783,0.796,0.731
svm_12_None=5_ros=1.2_h=0,0.735,0.726,0.783,0.775,0.802,0.726
svm_16_None=7_ros=1.2_h=0,0.733,0.723,0.781,0.772,0.799,0.723
svm_16_None=5_ros=1.2_h=0,0.733,0.721,0.781,0.769,0.802,0.721
mlp2_12_None=7_ros=1.2_h=0,0.729,0.725,0.775,0.779,0.782,0.776
mlp2_12_None=5_ros=1.2_h=0,0.729,0.724,0.775,0.777,0.784,0.776
svm_35_None=5_ros=1.2_h=0,0.728,0.721,0.778,0.781,0.787,0.721
rf_35_None=5_ros=1.2_h=0,0.728,0.713,0.787,0.762,0.829,0.796
rf_12_None=5_ros=1.2_h=0,0.726,0.717,0.779,0.763,0.813,0.783


**look at feat importances**

In [ ]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
# r = r[r.accuracy > np.percentile(r.accuracy, 80)]


keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
# r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)
r = r.reindex(r.iloc[0].sort_values(ascending=True).index, axis=1) # sort cols by mean rank
subset = list(r.keys())
subset.remove('acc')
r.insert(1, 'num_feats', r.apply(lambda row: np.sum(~np.isnan(row.values)) - 1, axis=1))


def color_vals(val):
    colors = {
        1: cb, 2: cp, 3: cr,
    }
    if not val in colors:
        color = 'maroon'
    else:
        color = colors[val]
    return 'color: %s' % color

r = r[r.num_feats <= 5].head(10)

r.fillna("").style.applymap(color_vals, subset=subset)
# r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored